## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Anaconda3\envs\testAI\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))  #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu
#classifier.add(Dropout(rate = 0.25))

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))


#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Anaconda3\envs\testAI\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/100
50000/50000 [==============================] - 60s 1ms/step - loss: 1.2490 - acc: 0.5679
Epoch 2/100
50000/50000 [==============================] - 59s 1ms/step - loss: 0.8047 - acc: 0.7174
Epoch 3/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.5768 - acc: 0.7987
Epoch 4/100
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3911 - acc: 0.8645
Epoch 5/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.2515 - acc: 0.9139
Epoch 6/100
50000/50000 [==============================] - 59s 1ms/step - loss: 0.1724 - acc: 0.9413
Epoch 7/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.1372 - acc: 0.9531
Epoch 8/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.1186 - acc: 0.9590
Epoch 9/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.1014 - acc: 0.9645
Epoch 10/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.0793 -

50000/50000 [==============================] - 58s 1ms/step - loss: 0.0133 - acc: 0.9960
Epoch 78/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.0136 - acc: 0.9960
Epoch 79/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0143 - acc: 0.9954
Epoch 80/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0100 - acc: 0.9970
Epoch 81/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0173 - acc: 0.9943
Epoch 82/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0161 - acc: 0.9951
Epoch 83/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.0131 - acc: 0.9961
Epoch 84/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.0108 - acc: 0.9966
Epoch 85/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0119 - acc: 0.9961
Epoch 86/100
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0089 - acc: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.1223639e-07, 6.1321043e-10, 5.1182724e-04, 9.2634398e-01,
        2.0171478e-04, 1.2683704e-09, 7.2917886e-02, 4.8366595e-09,
        2.3823124e-05, 7.1640881e-13]], dtype=float32)